In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
import sys
import os
import psycopg2 as psy
from psycopg2 import sql
from io import StringIO
import sqlalchemy
from sqlalchemy import create_engine
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/src")
import mtg_helpers as mtgh


con_details = {"dbname" : 'pay_to_play', 
               "user" : os.environ['PGSQL_P_USER'], 
               "password" : os.environ['PGSQL_P_PWD'], 
               "host" : 'localhost'} 


In [56]:
conn = mtgh.connect(con_details)

Connecting to PostgreSQL database...
Connection successful


### Let's start by exploring quickly the total card set data

In [57]:
query = sql.SQL(
    """SELECT 
            core.uuid, core.name, core.rarity, core."setCode", core."keywords", core."convertedManaCost", 
            core.type, core.power, core.toughness, core.subtypes, core.supertypes, 
            setdetails.setname, setdetails.setreleasedate, fixed_prices.price, fixed_prices."price_date"
    FROM core
    LEFT JOIN fixed_prices ON core.uuid = fixed_prices.uuid
    JOIN setdetails  ON core."setCode" = setdetails.setcode""")
card_df = pd.read_sql(query, conn)

In [58]:
card_df.describe()

,convertedManaCost,price
count,3705.000000,3536.000000
mean,3.043185,1.819762
std,1.841967,4.962194
min,0.000000,0.150000
25%,2.000000,0.250000
50%,3.000000,0.250000
75%,4.000000,0.990000
max,12.000000,109.990000


### This gives a nice view to the ranges of price and converted mana cost across all the cards in this analysis.

In [59]:
card_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3705 entries, 0 to 3704
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   uuid               3705 non-null   object 
 1   name               3705 non-null   object 
 2   rarity             3705 non-null   object 
 3   setCode            3705 non-null   object 
 4   keywords           1521 non-null   object 
 5   convertedManaCost  3705 non-null   float64
 6   type               3705 non-null   object 
 7   power              1850 non-null   object 
 8   toughness          1850 non-null   object 
 9   subtypes           3705 non-null   object 
 10  supertypes         3705 non-null   object 
 11  setname            3705 non-null   object 
 12  setreleasedate     3705 non-null   object 
 13  price              3536 non-null   float64
 14  price_date         3536 non-null   object 
dtypes: float64(2), object(13)
memory usage: 434.3+ KB


### Initial sense of the completness of each key column for potential analysis. Some 'gaps' are expected/explainable in that keywords, power, and toughness tend to only apply to creature-types and thus a ~50% null rate is to be expected.  Because of either the inherent nature of the data or custom behavior on some MTG cards, there are very few fields that can be of numeric types.  Of note would be the power and toughness columns as for creatures these are typically integers except for some creatures that have variable values based on the state of the game and are represented as asterisks.  Can still do numerical analysis on these fields by limiting to just those values with pre-defined integer values and casting to int type.  Biggest 'issue' will ultimately be around the ~150 cards that didn't have prices available from the target price source.

### Ultimately wanting to test our hypothesis that the world champion decks do not cost more than other successful, but amateur, decks.  So let's now look quickly at our deck data.

In [60]:
query = sql.SQL(
    """SELECT decklists.deckname, core.uuid, core.name, 
    decklists.card_count, decklists.sideboard, core.rarity, 
    core."setCode", core."keywords", core."convertedManaCost", 
    core.type, core.power, core.toughness, core.subtypes, core.supertypes, 
    setdetails.setname, setdetails.setreleasedate, fixed_prices.price, fixed_prices."price_date"
FROM core
LEFT JOIN fixed_prices ON core.uuid = fixed_prices.uuid
JOIN setdetails  ON core."setCode" = setdetails.setcode
JOIN decklists ON core.uuid = decklists.uuid""")
decks_df = pd.read_sql(query, conn)

In [61]:
decks_df.describe()

,card_count,convertedManaCost,price
count,1144.000000,1144.000000,1060.000000
mean,2.890734,2.311189,4.218528
std,1.968106,1.908038,7.551366
min,1.000000,0.000000,0.150000
25%,2.000000,1.000000,0.250000
50%,3.000000,2.000000,0.990000
75%,4.000000,3.000000,4.615000
max,20.000000,9.000000,64.990000


In [62]:
decks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1144 entries, 0 to 1143
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   deckname           1144 non-null   object 
 1   uuid               1144 non-null   object 
 2   name               1144 non-null   object 
 3   card_count         1144 non-null   int64  
 4   sideboard          1144 non-null   bool   
 5   rarity             1144 non-null   object 
 6   setCode            1144 non-null   object 
 7   keywords           377 non-null    object 
 8   convertedManaCost  1144 non-null   float64
 9   type               1144 non-null   object 
 10  power              320 non-null    object 
 11  toughness          320 non-null    object 
 12  subtypes           1144 non-null   object 
 13  supertypes         1144 non-null   object 
 14  setname            1144 non-null   object 
 15  setreleasedate     1144 non-null   object 
 16  price              1060 

### Unfortunately, the decision to limit pricing to a single source prevented complete acquisition of price data for each card.  Need to explore the perspective of deck data that only has prices and decide on an approach for moving forward.  Will compile a deck-based df that breaks out core vs. sideboard cards  and show card counts, total prices, and avg. price per card in each category.


In [90]:
price_only = decks_df.loc[pd.notnull(decks_df['price']), ['deckname', 'card_count', 'price', 'sideboard']]
price_only['total_card_price'] = price_only['card_count'] * price_only['price']
price_only.head()

,deckname,card_count,price,sideboard,total_card_price
0,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(19),4,0.25,False,1.00
1,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(1),3,0.25,True,0.75
2,Ondřej-Stráský-Azorius-Control-Magic-World-Cha...,2,0.25,True,0.50
3,Paulo-Vitor-Damo-da-Rosa-Azorius-Control-Magic...,2,0.25,True,0.50
4,Matias-Leveratto-Temur-Reclamation-Magic-World...,3,0.39,True,1.17


In [104]:
core_price_only = price_only.loc[price_only['sideboard'] == False,:]
core_deck_prices = core_price_only.loc[:,['deckname', 'card_count', 'total_card_price']].groupby('deckname').sum()
core_deck_prices['core_avg_card_price'] = core_deck_prices['total_card_price'] / core_deck_prices['card_count']
core_deck_prices.rename(columns = {})
core_deck_prices.rename(columns = {'card_count':'core_card_count','total_card_price':'core_deck_price'}, inplace = True)


sboard_price_only = price_only.loc[price_only['sideboard'] == True,:]
sboard_deck_prices = sboard_price_only.loc[:,['deckname', 'card_count', 'total_card_price']].groupby('deckname').sum()
sboard_deck_prices['sboard_avg_card_price'] = sboard_deck_prices['total_card_price'] / sboard_deck_prices['card_count']
sboard_deck_prices.rename(columns = {})
sboard_deck_prices.rename(columns = {'card_count':'sboard_card_count','total_card_price':'sboard_deck_price'}, inplace = True)

deck_prices = core_deck_prices.merge(sboard_deck_prices,how='inner',on='deckname')


In [106]:
deck_prices.describe()

,core_card_count,core_deck_price,core_avg_card_price,sboard_card_count,sboard_deck_price,sboard_avg_card_price
count,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
mean,51.659091,226.635227,4.503797,18.159091,46.366591,2.218371
std,14.496099,119.153386,2.563019,12.970348,80.441338,1.968854
min,1.000000,0.390000,0.390000,11.000000,5.230000,0.348667
25%,51.750000,141.425000,2.611726,15.000000,11.555000,0.770333
50%,56.000000,221.665000,4.046095,15.000000,26.490000,1.746000
75%,59.000000,312.352500,5.538869,15.000000,41.150000,2.840525
max,60.000000,445.140000,13.990000,68.000000,509.980000,8.499667


In [107]:
# create categorical variable for world comp vs. successful amateur, then look to compare sboard and core avg price
#within the categories.  maybe overlay histograms by avg card price bins?
# also then look at histograms of card count for core by each category...maybe as a pareto to make a 'cutoff'.
# Then plot the distributions of each select deck pricing group to compare the two.
# then do hypothesis test.
# ideally some scatters between price and other variables to see any kind of trend... and then pick sample from 
#core cards to create 'at random deck' to see what would generate.

In [10]:
def overlayed_plots(ax, df, category):
    for i in df[category].unique():
        ax.plot()

fig, ax = plt.subplots()
overlayed_plots(ax, card_df, 'rarity')





mythic
common
uncommon
rare


In [15]:
from pandas_profiling import ProfileReport

In [17]:
profile = ProfileReport(card_df, title='Pandas Profiling Report')

In [18]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/28 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]